In [1]:
# --------------------------------------------------
# 05_prepare_full_index.ipynb
# Convert all TCIA DICOMs → PNGs and build master index
# --------------------------------------------------

import os
from pathlib import Path
import pandas as pd
import pydicom
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
# --------------------------------------------------
# Step 1: Setup paths
# --------------------------------------------------
os.chdir(r"C:\Users\Negar\Desktop\paper_results\Myself\cr_coad_project")

RAW_DIR = Path("data/raw/tcia")
OUT_DIR = Path("data/processed/images/all_slices")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Clean existing directory
for old_file in OUT_DIR.glob("*.png"):
    old_file.unlink()


In [3]:
# --------------------------------------------------
# Step 2: Load clinical and metadata tables
# --------------------------------------------------
clinical_path = Path("data/processed/clinical/clinical_features_with_id.csv")
meta_path = Path("data/processed/clinical/tcia_series_metadata.csv")

if not clinical_path.exists() or not meta_path.exists():
    raise SystemExit("❌ Required input files missing. Please run 02_label_patients and 04_preprocess_wsi first.")

clinical_df = pd.read_csv(clinical_path)
meta_df = pd.read_csv(meta_path)

print(f"✅ Loaded clinical data: {len(clinical_df)} patients")
print(f"✅ Loaded imaging metadata: {len(meta_df)} series from {meta_df['patient_id'].nunique()} patients")


✅ Loaded clinical data: 633 patients
✅ Loaded imaging metadata: 521 series from 225 patients


In [4]:
# --------------------------------------------------
# Step 3: Merge clinical ↔ imaging metadata
# --------------------------------------------------
merged = meta_df.merge(clinical_df, on="patient_id", how="left")
print(f"🔗 Merged metadata shape: {merged.shape}")

# Log label coverage
label_coverage = merged["metastasis_status"].notna().mean() * 100
print(f"📊 Label coverage after merge: {label_coverage:.1f}% of series have metastasis labels")


🔗 Merged metadata shape: (521, 17)
📊 Label coverage after merge: 8.8% of series have metastasis labels


In [5]:
# --------------------------------------------------
# Step 4: Convert DICOMs → PNGs
# --------------------------------------------------
dicom_files = list(RAW_DIR.rglob("*.dcm"))
print(f"🔍 Found {len(dicom_files)} DICOM files under {RAW_DIR}")

records = []
bad_files = 0

for dcm_path in tqdm(dicom_files, desc="Converting DICOMs"):
    try:
        ds = pydicom.dcmread(dcm_path, force=True)
        img = ds.pixel_array

        # Normalize to 0–255
        img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
        img = img.astype("uint8")

        # Extract metadata
        patient_id = getattr(ds, "PatientID", None)
        series_id = getattr(ds, "SeriesInstanceUID", None)
        if patient_id is None or series_id is None:
            continue

        # Construct filename
        out_name = f"{patient_id}_{series_id}_{dcm_path.stem}.png"
        out_path = OUT_DIR / out_name
        cv2.imwrite(str(out_path), img)

        records.append({
            "patient_id": patient_id,
            "series_id": series_id,
            "slice_path": str(out_path).replace("\\", "/")
        })

    except Exception as e:
        bad_files += 1
        continue

print(f"✅ Successfully converted {len(records)} DICOM slices → PNGs")
print(f"⚠️ Skipped {bad_files} corrupted or unreadable files")


🔍 Found 28019 DICOM files under data\raw\tcia


Converting DICOMs: 100%|██████████| 28019/28019 [16:03<00:00, 29.07it/s] 

✅ Successfully converted 27822 DICOM slices → PNGs
⚠️ Skipped 197 corrupted or unreadable files


In [6]:
# --------------------------------------------------
# Step 5: Build image index
# --------------------------------------------------
img_df = pd.DataFrame(records)
img_df = img_df.drop_duplicates(subset=["slice_path"])

# Merge to include clinical labels
index_df = img_df.merge(merged, on=["patient_id", "series_id"], how="left")

print(f"✅ Final index built: {len(index_df)} total image slices")
print(f"📊 Patients represented: {index_df['patient_id'].nunique()}")
print(f"📊 Series represented: {index_df['series_id'].nunique()}")


✅ Final index built: 27822 total image slices
📊 Patients represented: 225
📊 Series represented: 324


In [7]:
# --------------------------------------------------
# Step 6: Save index file safely
# --------------------------------------------------
out_index = Path("data/processed/images/all_index.csv")
if out_index.exists():
    out_index.unlink()

index_df.to_csv(out_index, index=False)
print(f"✅ Saved clean image index to: {out_index}")


✅ Saved clean image index to: data\processed\images\all_index.csv


In [8]:
# --------------------------------------------------
# Step 7: Create new train/val/test splits (series-level)
# --------------------------------------------------
series_ids = index_df["series_id"].dropna().unique()
train_ids, temp_ids = train_test_split(series_ids, test_size=0.3, random_state=42)
val_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)

split_dir = Path("data/splits")
split_dir.mkdir(parents=True, exist_ok=True)

pd.DataFrame({"series_id": train_ids}).to_csv(split_dir / "train_series.csv", index=False)
pd.DataFrame({"series_id": val_ids}).to_csv(split_dir / "val_series.csv", index=False)
pd.DataFrame({"series_id": test_ids}).to_csv(split_dir / "test_series.csv", index=False)

print("✅ Train/Val/Test splits created under data/splits/")
print(f"   ➤ Train: {len(train_ids)} series")
print(f"   ➤ Val:   {len(val_ids)} series")
print(f"   ➤ Test:  {len(test_ids)} series")


✅ Train/Val/Test splits created under data/splits/
   ➤ Train: 226 series
   ➤ Val:   49 series
   ➤ Test:  49 series


In [10]:
# --------------------------------------------------
# Step 8: Diagnostics summary
# --------------------------------------------------
print("\n📋 FINAL SUMMARY")
print(f"🧩 Total image slices: {len(index_df)}")
print(f"🧍 Unique patients: {index_df['patient_id'].nunique()}")
print(f"🩻 Unique series: {index_df['series_id'].nunique()}")
print(f"🏷️ Labeled slices: {index_df['metastasis_status'].notna().sum()} ({index_df['metastasis_status'].notna().mean()*100:.1f}%)")
print(f"🧾 Index saved to: {out_index}")


📋 FINAL SUMMARY
🧩 Total image slices: 27822
🧍 Unique patients: 225
🩻 Unique series: 324
🏷️ Labeled slices: 3408 (12.2%)
🧾 Index saved to: data\processed\images\all_index.csv
